In [2]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')



# Functions to analyze files csv


In [3]:
#Extrai o valor de perdas totais de um arquivo csv e retorna um dataframe com os valores
def sum_TotalLosses(file_csv):
    df = pd.read_csv(os.path.join(file_csv))
    columns = ['P(kW)','Q(kvar)','TotalLosses(kva)']
    P = round(df['P(kW)'].sum(),4)
    Q = round(df['Q(kvar)'].sum(),4)
    S = round(np.sqrt(P**2 + Q**2),4)
    new_row = pd.DataFrame([[P,Q,S]],columns=columns)
    return new_row

#Extrai os valores máximos e minímos de tensão nos barramentos do arquivo .csv e retorna um dataframe com os valores
def max_min_Voltage(file_csv):
    df = pd.read_csv(os.path.join(file_csv))
    columns = ['Bus','Vmax(pu)','Vmin(pu)']
    #Lista todos os barramentos diferentes
    bus = df['Bus'].unique()
    Vmax = []
    Vmin = []
    for i in bus:
        Vmax.append(df[df['Bus']==i]['Voltage (p.u.)'].max())
        Vmin.append(df[df['Bus']==i]['Voltage (p.u.)'].min())
    new_row = pd.DataFrame(list(zip(bus,Vmax,Vmin)),columns=columns)
    return new_row

# Extrai os valores de potência dos barramentos do arquivo .csv e retorna um dataframe com os valores
def extract_BusPowers(file_csv):
    df = pd.read_csv(os.path.join(file_csv))
    columns = ['Bus','P(kW)','Pmax','Q(kvar)','Qmax','S(kva)','Smax']	
    #Lista todos os barramentos diferentes
    bus = df['Bus'].unique()
    P = []
    MaxP = []
    Q = []
    MaxQ = []
    S = []
    MaxS = []
    # Para cada barramento extrai os valores de potência
    for i in bus:
        P.append(df[df['Bus']==i]['P(kW)'].sum())
        MaxP.append(df[df['Bus']==i]['P(kW)'].max())
        Q.append(df[df['Bus']==i]['Q(kvar)'].sum())
        MaxQ.append(df[df['Bus']==i]['Q(kvar)'].max())
        S.append(np.sqrt(P[-1]**2 + Q[-1]**2))
        MaxS.append(np.sqrt(MaxP[-1]**2 + MaxQ[-1]**2))
    # Cria o dataframe
    new_row = pd.DataFrame(list(zip(bus,P,MaxP,Q,MaxQ,S,MaxS)),columns=columns)
    return new_row

#Extrai os valores de demanda do circuito
def extract_demand(file_csv):
    df = pd.read_csv(os.path.join(file_csv))
    columns = ['PmaxIn','PminIn','PmaxOut','PminOut','QmaxIn','QminIn','QmaxOut','QminOut','SmaxIn','SminIn','SmaxOut','SminOut']

    # Filtra somente os valores positivos de potência que significa potência entrando no circuito
    Pin = df[df['P(kW)'] > 0]['P(kW)']
    PmaxIn = Pin.max()
    PminIn = Pin.min()
    Qin = df[df['Q(kvar)'] > 0]['Q(kvar)']
    QmaxIn = Qin.max()
    QminIn = Qin.min()
    #Calcula SmaxIn e SminIn
    SmaxIn = np.sqrt(PmaxIn**2+QmaxIn**2)
    SminIn = np.sqrt(PminIn**2+QminIn**2)

    #Filtra somente os valores negativos que significa potência no sentido contrário, sendo injetado na rede
    Pout = df[df['P(kW)'] < 0]['P(kW)']
    PmaxOut = Pout.min()
    PminOut = Pout.max()
    # Extrai os valores de Q(kvar) correspondentes a PmaxOut e PminOut
    QmaxOut = df[df['P(kW)'] == PmaxOut]['Q(kvar)'].values[0] 
    QminOut = df[df['P(kW)'] == PminOut]['Q(kvar)'].values[0] 
    
    #Calcula SmaxOut e SminOut
    SmaxOut = -np.sqrt(PmaxOut**2+QmaxOut**2)
    SminOut = -np.sqrt(PminOut**2+QminOut**2)
    # Cria o dataframe
    new_row = pd.DataFrame([[PmaxIn,PminIn,PmaxOut,PminOut,QmaxIn,QminIn,QmaxOut,QminOut,SmaxIn,SminIn,SmaxOut,SminOut]],columns=columns)
    return new_row

#Extrai os valores de carga do circuito
def extrac_load(file_csv):
    df = pd.read_csv(os.path.join(file_csv))
    columns = ['Pmax','Pmin','Qmax','Qmin','Smax','Smin']
    Pmax = df['P(kW)'].max()
    Pmin = df['P(kW)'].min()
    Qmax = df['Q(kvar)'].max()
    Qmin = df['Q(kvar)'].min()
    Smax = np.sqrt(Pmax**2 + Qmax**2)
    Smin = np.sqrt(Pmin**2 + Qmin**2)
    new_row = pd.DataFrame([[Pmax,Pmin,Qmax,Qmin,Smax,Smin]],columns=columns)
    return new_row
    

# Calcula o indice de suavização da curva de demanda
def smoothness_index(file_csv):
    # Extrai o nome do arquivo
    name_file = file_csv.split('/')[-1]
    bus = name_file.split('_')[2]
    #Calcula as derivadas da curva de demanda
    df = pd.read_csv(os.path.join(file_csv))
    df_dt = np.gradient(df['P(kW)'])
    df2_dt2 = np.gradient(df_dt)
    #Calcula os indices de suavização
    smoothens_std = round(np.std(df_dt),4)
    norm_smoothness = round(np.sum(df2_dt2**2),4)

    # Cria o dataframe
    new_row = pd.DataFrame([[bus,smoothens_std,norm_smoothness]],columns=['Bus','Smoothness_std','Norm_smoothness'])
    return new_row
      


# Analise dos dados gerados

In [4]:
# Input and output paths    
path_load = os.getcwd() + '/data/loads/'
path_xlsx = os.getcwd() + '/data/spreadsheets/'
path_dss = os.getcwd() + '/data/dss_files/'	
output_csv = os.getcwd() + '/data/output/csv/'

path_branch = output_csv + 'branch_flows/'
path_busPower = output_csv + 'bus_power/'
path_busVoltage = output_csv + 'bus_voltage/'
path_demand = output_csv + 'demand/'
path_load = output_csv + 'load/'
path_losses = output_csv + 'losses/'

files_dss = ['5Nodeckt','CondominioDosIpes']
dados_analise = ['BranchFlow','BusPowers','BusVoltage','Demand','Load','Losses']


# Analise dos dados sem operação do BESS

In [5]:
files_BranchFlow_NoOperation = ['BranchFlow_NoOperation_year_5Nodeckt.csv','BranchFlow_NoOperation_year_CondominioDosIpes.csv']
files_BusPowers_NoOperation = ['BusPowers_NoOperation_year_5Nodeckt.csv','BusPowers_NoOperation_year_CondominioDosIpes.csv']
files_BusVoltage_NoOperation = ['BusVoltage_NoOperation_year_5Nodeckt.csv','BusVoltage_NoOperation_year_CondominioDosIpes.csv']
files_Demand_NoOperation = ['Demand_NoOperation_year_5Nodeckt.csv','Demand_NoOperation_year_CondominioDosIpes.csv']
files_Load_NoOperation = ['Load_NoOperation_year_5Nodeckt.csv','Load_NoOperation_year_CondominioDosIpes.csv']
files_Losses_NoOperation = ['Losses_NoOperation_year_5Nodeckt.csv','Losses_NoOperation_year_CondominioDosIpes.csv']


In [6]:
from IPython.display import display, Markdown

display(Markdown('**Analisando as Perdas totais:**\n'))
for file in files_Losses_NoOperation:
    topografia = file.split('.')[0].split('_')[3]
    print('Topografia do circuito: ',topografia)
    display(sum_TotalLosses(os.path.join(path_losses,file)))

display(Markdown('**Analisando as tensões dos barramentos:**\n'))
for file in files_BusVoltage_NoOperation:
    topografia = file.split('.')[0].split('_')[3]
    print('Topografia do circuito: ',topografia)
    display(max_min_Voltage(os.path.join(path_busVoltage,file)))

display(Markdown('**Analisando as potências dos barramentos:**\n'))
for file in files_BusPowers_NoOperation:
    topografia = file.split('.')[0].split('_')[3]
    print('Topografia do circuito: ',topografia)
    display(extract_BusPowers(os.path.join(path_busPower,file)))

display(Markdown('**Analisando as demandas do circuito:**\n'))
for file in files_Demand_NoOperation:
    topografia = file.split('.')[0].split('_')[3]
    print('Topografia do circuito: ',topografia)
    display(extract_demand(os.path.join(path_demand,file)))

display(Markdown('**Analisando as cargas do circuito:**\n'))
for file in files_Load_NoOperation:
    topografia = file.split('.')[0].split('_')[3]
    print('Topografia do circuito: ',topografia)
    display(extrac_load(os.path.join(path_load,file)))

**Analisando as Perdas totais:**


Topografia do circuito:  5Nodeckt


,P(kW),Q(kvar),TotalLosses(kva)
0,5.0733,5.547,7.5172


Topografia do circuito:  CondominioDosIpes


,P(kW),Q(kvar),TotalLosses(kva)
0,8100.7577,183.5712,8102.8374


...

**Analisando as cargas do circuito:**


Topografia do circuito:  5Nodeckt


,Pmax,Pmin,Qmax,Qmin,Smax,Smin
0,2.2765,0.1468,0.753,0.0585,2.397803,0.158027


Topografia do circuito:  CondominioDosIpes


,Pmax,Pmin,Qmax,Qmin,Smax,Smin
0,11.1984,1.9167,4.2584,0.7496,11.98074,2.058067


# Analisando em qual Barramento para o modo de Operação Simple teve menor perda na topologia da rede de 5 nós

In [7]:
files_simple = [file for file in os.listdir(path_losses) if 'Simple' in file]
files_simple = [file for file in files_simple if '5Nodeckt' in file]
print(files_simple)
colums = ['Barramento','P(kW)','Q(kvar)','TotalLosses(kva)']
df = pd.DataFrame(columns=colums)
for file in files_simple:
    barramento = file.split('.')[0].split('_')[2]
    losses = sum_TotalLosses(os.path.join(path_losses,file))
    losses['Barramento'] = barramento
    df = pd.concat([df,losses],ignore_index=True)
display(df)
#Extract the min value of P 
P = df['P(kW)'].min()
#Extract all values from the row with the min value of P
row = df[df['P(kW)'] == P]
print('Barramento com a menor perda: ')
print(row)

['Losses_Simple_bus001_year_5Nodeckt.csv', 'Losses_Simple_bus002_year_5Nodeckt.csv', 'Losses_Simple_bus003_year_5Nodeckt.csv', 'Losses_Simple_bus004_year_5Nodeckt.csv', 'Losses_Simple_bus005_year_5Nodeckt.csv']


,Barramento,P(kW),Q(kvar),TotalLosses(kva)
0,bus001,5.0733,5.5470,7.5172
1,bus002,5.1992,5.6556,7.6823
2,bus003,5.3451,5.7845,7.8759
3,bus004,5.5006,5.9184,8.0799
4,bus005,5.7097,6.1001,8.3554


Barramento com a menor perda: 
  Barramento   P(kW)  Q(kvar)  TotalLosses(kva)
0     bus001  5.0733    5.547            7.5172


# Analisando em qual Barramento para o modo de Operação Smoothing teve menor perda na topologia da rede de 5 nós

In [8]:
file_smoothing = [file for file in os.listdir(path_losses) if 'Smoothing' in file]
file_smoothing = [file for file in file_smoothing if '5Nodeckt' in file]
print(file_smoothing)
colums = ['Barramento','P(kW)','Q(kvar)','TotalLosses(kva)']
df = pd.DataFrame(columns=colums)
for file in file_smoothing:
    barramento = file.split('.')[0].split('_')[2]
    losses = sum_TotalLosses(os.path.join(path_losses,file))
    losses['Barramento'] = barramento
    df = pd.concat([df,losses],ignore_index=True)
display(df)
#Extract the min value of P 
P = df['P(kW)'].min()
#Extract all values from the row with the min value of P
row = df[df['P(kW)'] == P]
print('Barramento com a menor perda: ')
print(row)

['Losses_Smoothing_bus001_year_5Nodeckt.csv', 'Losses_Smoothing_bus002_year_5Nodeckt.csv', 'Losses_Smoothing_bus003_year_5Nodeckt.csv', 'Losses_Smoothing_bus004_year_5Nodeckt.csv', 'Losses_Smoothing_bus005_year_5Nodeckt.csv']


,Barramento,P(kW),Q(kvar),TotalLosses(kva)
0,bus001,5.0733,5.5470,7.5172
1,bus002,5.0877,5.5625,7.5383
2,bus003,5.1107,5.5865,7.5715
3,bus004,5.1243,5.5852,7.5798
4,bus005,5.1343,5.5956,7.5942


Barramento com a menor perda: 
  Barramento   P(kW)  Q(kvar)  TotalLosses(kva)
0     bus001  5.0733    5.547            7.5172


# Analise dos dados da Topologia CondominioDosIpes modo de Operação Simple

In [9]:
files_simple = [file for file in os.listdir(path_losses) if 'Simple' in file]
files_simple = [file for file in files_simple if 'CondominioDosIpes' in file]
print(files_simple)
colums = ['Barramento','P(kW)','Q(kvar)','TotalLosses(kva)']
df = pd.DataFrame(columns=colums)
for file in files_simple:
    barramento = file.split('.')[0].split('_')[2]
    losses = sum_TotalLosses(os.path.join(path_losses,file))
    losses['Barramento'] = barramento
    df = pd.concat([df,losses],ignore_index=True)
display(df)
#Extract the min value of P 
P = df['P(kW)'].min()
#Extract all values from the row with the min value of P
row = df[df['P(kW)'] == P]
print('Barramento com a menor perda: ')
print(row)

['Losses_Simple_bus001_year_CondominioDosIpes.csv', 'Losses_Simple_bus002_year_CondominioDosIpes.csv', 'Losses_Simple_bus003_year_CondominioDosIpes.csv', 'Losses_Simple_bus004_year_CondominioDosIpes.csv', 'Losses_Simple_bus005_year_CondominioDosIpes.csv', 'Losses_Simple_bus006_year_CondominioDosIpes.csv', 'Losses_Simple_bus007_year_CondominioDosIpes.csv', 'Losses_Simple_bus008_year_CondominioDosIpes.csv', 'Losses_Simple_bus009_year_CondominioDosIpes.csv', 'Losses_Simple_bus010_year_CondominioDosIpes.csv', 'Losses_Simple_bus011_year_CondominioDosIpes.csv', 'Losses_Simple_bus012_year_CondominioDosIpes.csv', 'Losses_Simple_bus013_year_CondominioDosIpes.csv', 'Losses_Simple_bus014_year_CondominioDosIpes.csv']


,Barramento,P(kW),Q(kvar),TotalLosses(kva)
0,bus001,8086.6464,143.5615,8087.9206
1,bus002,8073.1483,137.9534,8074.3269
2,bus003,8098.1485,148.4004,8099.5081
3,bus004,8126.7416,160.3379,8128.3232
4,bus005,8151.8441,168.2041,8153.5793
5,bus006,8182.2148,180.8777,8184.2138
6,bus007,8048.2146,127.6210,8049.2264
7,bus008,8054.9129,130.4194,8055.9687
8,bus009,8063.9522,134.1359,8065.0677
9,bus010,8057.0618,131.2938,8058.1315


Barramento com a menor perda: 
  Barramento      P(kW)  Q(kvar)  TotalLosses(kva)
6     bus007  8048.2146  127.621         8049.2264


# Analise dos dados da Topologia CondominioDosIpes modo de Operação Smoothing

In [10]:
files_simple = [file for file in os.listdir(path_losses) if 'Smoothing' in file]
files_simple = [file for file in files_simple if 'CondominioDosIpes' in file]
print(files_simple)
colums = ['Barramento','P(kW)','Q(kvar)','TotalLosses(kva)']
df = pd.DataFrame(columns=colums)
for file in files_simple:
    barramento = file.split('.')[0].split('_')[2]
    losses = sum_TotalLosses(os.path.join(path_losses,file))
    losses['Barramento'] = barramento
    df = pd.concat([df,losses],ignore_index=True)
display(df)
#Extract the min value of P 
P = df['P(kW)'].min()
#Extract all values from the row with the min value of P
row = df[df['P(kW)'] == P]
print('Barramento com a menor perda: ')
print(row)

['Losses_Smoothing_bus001_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus002_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus003_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus004_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus005_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus006_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus007_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus008_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus009_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus010_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus011_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus012_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus013_year_CondominioDosIpes.csv', 'Losses_Smoothing_bus014_year_CondominioDosIpes.csv']


,Barramento,P(kW),Q(kvar),TotalLosses(kva)
0,bus001,8098.7311,179.0979,8100.7112
1,bus002,8096.4639,178.1547,8098.4237
2,bus003,8098.0741,178.8241,8100.0483
3,bus004,8099.1819,179.2832,8101.1660
4,bus005,8098.6068,178.9326,8100.5832
5,bus006,8099.9054,179.4772,8101.8936
6,bus007,8091.0909,175.9260,8093.0033
7,bus008,8090.7377,175.7797,8092.6470
8,bus009,8094.8844,177.5062,8096.8304
9,bus010,8093.3371,176.8569,8095.2692


Barramento com a menor perda: 
   Barramento      P(kW)  Q(kvar)  TotalLosses(kva)
10     bus011  8090.4047  175.636         8092.3109


# Indices de suavização sem operação do bess

In [11]:
display(Markdown('**Analisando indice de suavização sem operação:**\n'))
for file in files_Demand_NoOperation:
    topografia = file.split('.')[0].split('_')[3]
    print('Topografia do circuito: ',topografia)
    display(smoothness_index(os.path.join(path_demand,file)))

**Analisando indice de suavização sem operação:**


Topografia do circuito:  5Nodeckt


,Bus,Smoothness_std,Norm_smoothness
0,year,0.1351,155.3546


Topografia do circuito:  CondominioDosIpes


,Bus,Smoothness_std,Norm_smoothness
0,year,1.1235,8049.1656


# Analisar sob vista do ponto de suavização qual barramento teve melhor indice no diagrama 5Nodeckt

In [12]:
files_simple = [file for file in os.listdir(path_demand) if 'Simple' in file]
files_simple = [file for file in files_simple if '5Nodeckt' in file]

colums = ['Bus','Smoothness_std','Norm_smoothness']
df = pd.DataFrame(columns=colums)
display(Markdown('**Analisando indice de suavização modo de operação Simple 5Nodeckt:**\n'))
for file in files_simple:
    df = pd.concat([df,smoothness_index(os.path.join(path_demand,file))],ignore_index=True)

display(df)
min_std = df['Smoothness_std'].min()
row = df[df['Smoothness_std'] == min_std]
print('Barramento com menor desvio padrão: ')
print(row)
min_norm = df['Norm_smoothness'].min()
row = df[df['Norm_smoothness'] == min_norm]
print('Barramento com menor norma: ')
print(row)

print('')

colums = ['Bus','Smoothness_std','Norm_smoothness']
df = pd.DataFrame(columns=colums)

display(Markdown('**Analisando indice de suavização modo de operação Smoothing 5Nodeckt:**\n'))
files_smoothing = [file for file in os.listdir(path_demand) if 'Smoothing' in file]
files_smoothing = [file for file in files_smoothing if '5Nodeckt' in file]
for file in files_smoothing:
    df = pd.concat([df,smoothness_index(os.path.join(path_demand,file))],ignore_index=True)

display(df)
min_std = df['Smoothness_std'].min()
row = df[df['Smoothness_std'] == min_std]
print('Barramento com menor desvio padrão: ')
print(row)
min_norm = df['Norm_smoothness'].min()
row = df[df['Norm_smoothness'] == min_norm]
print('Barramento com menor norma: ')
print(row)


**Analisando indice de suavização modo de operação Simple 5Nodeckt:**


,Bus,Smoothness_std,Norm_smoothness
0,bus001,0.1340,162.2749
1,bus002,0.1340,162.2929
2,bus003,0.1340,162.3339
3,bus004,0.1340,162.2930
4,bus005,0.1341,162.3446


Barramento com menor desvio padrão: 
      Bus  Smoothness_std  Norm_smoothness
0  bus001           0.134         162.2749
1  bus002           0.134         162.2929
2  bus003           0.134         162.3339
3  bus004           0.134         162.2930
Barramento com menor norma: 
      Bus  Smoothness_std  Norm_smoothness
0  bus001           0.134         162.2749



**Analisando indice de suavização modo de operação Smoothing 5Nodeckt:**


,Bus,Smoothness_std,Norm_smoothness
0,bus001,0.1088,66.0161
1,bus002,0.1087,65.9112
2,bus003,0.1086,65.6038
3,bus004,0.1087,66.0039
4,bus005,0.1088,66.0405


Barramento com menor desvio padrão: 
      Bus  Smoothness_std  Norm_smoothness
2  bus003          0.1086          65.6038
Barramento com menor norma: 
      Bus  Smoothness_std  Norm_smoothness
2  bus003          0.1086          65.6038


# Análise da suavização no diagrama CondominioDosIpes

In [13]:
files_simple = [file for file in os.listdir(path_demand) if 'Simple' in file]
files_simple = [file for file in files_simple if 'CondominioDosIpes' in file]

colums = ['Bus','Smoothness_std','Norm_smoothness']
df = pd.DataFrame(columns=colums)

display(Markdown('**Analisando indice de suavização modo de operação Simple CondominioDosIpes:**\n'))
for file in files_simple:
    df = pd.concat([df,smoothness_index(os.path.join(path_demand,file))],ignore_index=True)

display(df)
min_std = df['Smoothness_std'].min()
row = df[df['Smoothness_std'] == min_std]
print('Barramento com menor desvio padrão: ')
print(row)
min_norm = df['Norm_smoothness'].min()
row = df[df['Norm_smoothness'] == min_norm]
print('Barramento com menor norma: ')
print(row)
print('')

colums = ['Bus','Smoothness_std','Norm_smoothness']
df = pd.DataFrame(columns=colums)

display(Markdown('**Analisando indice de suavização modo de operação Smoothing CondominioDosIpes:**\n'))
files_smoothing = [file for file in os.listdir(path_demand) if 'Smoothing' in file]
files_smoothing = [file for file in files_smoothing if 'CondominioDosIpes' in file]
for file in files_smoothing:
    df = pd.concat([df,smoothness_index(os.path.join(path_demand,file))],ignore_index=True)

display(df)
min_std = df['Smoothness_std'].min()
row = df[df['Smoothness_std'] == min_std]
print('Barramento com menor desvio padrão: ')
print(row)
min_norm = df['Norm_smoothness'].min()
row = df[df['Norm_smoothness'] == min_norm]
print('Barramento com menor norma: ')
print(row)


**Analisando indice de suavização modo de operação Simple CondominioDosIpes:**


,Bus,Smoothness_std,Norm_smoothness
0,bus001,1.1110,10749.7857
1,bus002,1.1106,10739.2826
2,bus003,1.1105,10737.9116
3,bus004,1.1103,10732.7252
4,bus005,1.1099,10723.5407
5,bus006,1.1099,10721.4150
6,bus007,1.1092,10707.9874
7,bus008,1.1090,10702.3293
8,bus009,1.1103,10731.8831
9,bus010,1.1098,10722.6324


Barramento com menor desvio padrão: 
       Bus  Smoothness_std  Norm_smoothness
13  bus014          1.1084       10689.1171
Barramento com menor norma: 
       Bus  Smoothness_std  Norm_smoothness
13  bus014          1.1084       10689.1171



**Analisando indice de suavização modo de operação Smoothing CondominioDosIpes:**


,Bus,Smoothness_std,Norm_smoothness
0,bus001,0.9607,3812.5854
1,bus002,0.9608,3815.0231
2,bus003,0.9608,3813.7803
3,bus004,0.9607,3813.3557
4,bus005,0.9608,3814.8739
5,bus006,0.9608,3814.2055
6,bus007,0.9611,3820.9858
7,bus008,0.9611,3821.6372
8,bus009,0.9609,3816.6929
9,bus010,0.9609,3818.3873


Barramento com menor desvio padrão: 
       Bus  Smoothness_std  Norm_smoothness
0   bus001          0.9607        3812.5854
3   bus004          0.9607        3813.3557
12  bus013          0.9607        3811.9074
Barramento com menor norma: 
       Bus  Smoothness_std  Norm_smoothness
12  bus013          0.9607        3811.9074
